## Canada Postal Codes Dataframe Assignment

### Importing Libraries

In [9]:
import pandas as pd # library for data analsysis
import numpy as np

from bs4 import BeautifulSoup as bs # Library for Web scraping
import urllib

import warnings

print('Libraries imported')

Libraries imported


### Creating Soup object to store web site data

In [10]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

soup = bs(urllib.request.urlopen(URL), 'html.parser')
print(soup.prettify()[:500])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_po


### Extracting column & row data to create dataframe

In [11]:
#Extracting column names (Table headers) for pandas dataframe
columns = [str(e.string.replace("\n","")) for e in soup.table.find_all('th')] 
print(columns)

['Postcode', 'Borough', 'Neighbourhood']


In [12]:
# Extracting row data for pandas dataframe
row_text = [str(bs(str(row).replace("\n",""), "lxml").text) for row in soup.table.find_all('td')]
row_text[:20]

['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A',
 'North York',
 'Victoria Village',
 'M5A',
 'Downtown Toronto',
 'Harbourfront',
 'M5A',
 'Downtown Toronto',
 'Regent Park',
 'M6A',
 'North York']

In [13]:
postcodes = row_text[0::3]
boroughs = row_text[1::3]
neighbourhoods =  row_text[2::3]

print("Length: {},{},{}".format(len(postcodes),len(boroughs),len(neighbourhoods)))

Length: 289,289,289


### Tranform the data into a *pandas* dataframe

In [14]:
warnings.filterwarnings('ignore')

df = pd.DataFrame(columns=columns)

# Looping thorugh lists created above to insert data into dataframe
df.Postcode = postcodes
df.Borough =  boroughs
df.Neighbourhood = neighbourhoods
print("df dimensions:",df.shape)

# Applying filters to get relevant data 

# Filter mask to remove "not assigned" Boroughs
filter_mask = df.Borough == "Not assigned"

#Filtering dataframe
df_filtered = df[~filter_mask]

#Replacing Neighborhoods with "not assigned" with Borough value:
df_filtered.Neighbourhood[df_filtered.Neighbourhood == 'Not assigned'] = df_filtered.Borough
print("df_filtered dimensions:",df_filtered.shape)

df dimensions: (289, 3)
df_filtered dimensions: (212, 3)


In [15]:
df_final = df_filtered.groupby(['Postcode','Borough']).Neighbourhood.agg(lambda val: ','.join(val)).reset_index()
df_final.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Displaying number of rows in final dataset

In [16]:
print(df_final.shape)

(103, 3)
